# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
DEVICE = torch.device('cpu')
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
   def __init__(self):
       super().__init__()
       self.conv1 = nn.Conv2d(1, 10, kernel_size=(6,6))
       self.conv2 = nn.Conv2d(10, 20, kernel_size=(2,2))
       self.linear1 = nn.Linear(20*10*10, 500)
       self.linear2 = nn.Linear(500, 10)

   def forward(self,batch):
       batch_num = batch.size(0)
       mid = self.conv1(batch)
       mid = torch.nn.functional.relu(mid)
       mid = torch.nn.functional.max_pool2d(mid, (2, 2))
       mid = self.conv2(mid)
       mid = torch.nn.functional.relu(mid)
       mid = mid.view(batch_num, -1)
       mid = self.linear1(mid)
       mid = torch.nn.functional.relu(mid)
       mid = self.linear2(mid)
       out = torch.nn.functional.log_softmax(mid, dim=1)
       return out

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = torch.nn.functional.nll_loss
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    print('epoch: ',epoch+1)
    model.train()
    for images, labels in train_loader:
        # TODO:forward + backward + optimize
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    test_loss = 0
    correct_num_test = 0
    train_loss = 0
    correct_num_train = 0
    with torch.no_grad():
       for images, labels in train_loader:
           images, labels = images.to(DEVICE), labels.to(DEVICE)
           output = model(images)
           train_loss += torch.nn.functional.nll_loss(output, labels, reduction='sum').item()
           pred = output.max(1, keepdim=True)[1]
           correct_num_train += pred.eq(labels.view_as(pred)).sum().item()
    train_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_train / len(train_loader.dataset)
    print('Train Accuracy: {0}/{1}({2:.4}%)'.format(correct_num_train,len(train_loader.dataset),correct_rate))
    
    with torch.no_grad():
       for images, labels in test_loader:
           images, labels = images.to(DEVICE), labels.to(DEVICE)
           output = model(images)
           test_loss += torch.nn.functional.nll_loss(output, labels, reduction='sum').item()
           pred = output.max(1, keepdim=True)[1]
           correct_num_test += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_test / len(test_loader.dataset)
    print('Test  Accuracy: {0}/{1}({2:.4}%)'.format(correct_num_test,len(test_loader.dataset),correct_rate))

print('Training and Testing Finished!')

epoch:  1
Train Accuracy: 58597/60000(97.66%)
Test  Accuracy: 9769/10000(97.69%)
epoch:  2
Train Accuracy: 59201/60000(98.67%)
Test  Accuracy: 9838/10000(98.38%)
epoch:  3
Train Accuracy: 59501/60000(99.17%)
Test  Accuracy: 9863/10000(98.63%)
epoch:  4
Train Accuracy: 59599/60000(99.33%)
Test  Accuracy: 9871/10000(98.71%)
epoch:  5
Train Accuracy: 59554/60000(99.26%)
Test  Accuracy: 9851/10000(98.51%)
epoch:  6
Train Accuracy: 59722/60000(99.54%)
Test  Accuracy: 9875/10000(98.75%)
epoch:  7
Train Accuracy: 59582/60000(99.3%)
Test  Accuracy: 9860/10000(98.6%)
epoch:  8
Train Accuracy: 59812/60000(99.69%)
Test  Accuracy: 9890/10000(98.9%)
epoch:  9
Train Accuracy: 59786/60000(99.64%)
Test  Accuracy: 9886/10000(98.86%)
epoch:  10
Train Accuracy: 59762/60000(99.6%)
Test  Accuracy: 9883/10000(98.83%)
Training and Testing Finished!
